In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
comb_cost_df_19_23 = pd.read_csv(r"C:\Users\Vivek.Sujan\Desktop\DS Assignment\Fuzzy K means clustering $0 and non $0 Premium\Final_data_v19.csv")

In [3]:
comb_cost_df_24 = pd.read_csv(r"C:\Users\Vivek.Sujan\Desktop\DS Assignment\Fuzzy K means clustering $0 and non $0 Premium\Final_data_2024_v5_DS.csv")

In [4]:
bene_weights = pd.read_excel(r"C:\Users\Vivek.Sujan\Desktop\DS Assignment\Plan Segmentation 2024 Plans\Benefit Weights\Benefit_Weights.xlsx")

In [5]:
excluded_values = ['Special Needs Plan Type_Chronic or Disabling Condition',
                   'Special Needs Plan Type_Institutional',
                   'Supplemental Benefit Value', 
                   'HWAI True Plan Value Part C', 
                   'HWAI True Plan Value Part D', 
                   'HWAI True Plan Value Total', 
                   'Actuarial (%)']
bene_weights = bene_weights[~bene_weights['Benefit_Names'].isin(excluded_values)]

In [6]:
bene_weights

,Benefit_Names,Benefit_Weights,Benefit_Weights_New,Benefit_Weights_New_1,Benefit_Weights_New_2,Benefit_Weights_New_3,Benefit_Weights_New_4,Benefit_Weights_New_5
0,Monthly_Consolidated_Premium_C_D,10.00,5,5.0,2.50,2.0,5.0,5.0
1,PartC_Premium,10.00,5,5.0,2.50,2.0,5.0,5.0
2,PartD_Premium,10.00,5,5.0,2.50,2.0,2.0,2.0
3,$0_Premium,10.00,5,5.0,2.50,2.0,5.0,5.0
4,Grand_Total_OOPC,5.00,3,4.0,2.25,1.8,2.0,2.0
...,...,...,...,...,...,...,...,...
190,b9c_Substance_Abuse_OON,0.45,1,0.8,0.80,1.0,1.0,1.0
191,b9d_Outpatient_Blood_Services_OON,0.45,1,0.8,0.80,1.0,1.0,1.0
192,b1a_inpat_acute_OON,0.45,1,0.8,0.80,1.0,1.0,1.0
193,b1b_inpat_mental_OON,0.45,1,0.8,0.80,1.0,1.0,1.0


In [11]:
cols = ['bid_id','Year'] + list(bene_weights['Benefit_Names']) 

In [12]:
comb_cost_df_19_23 = comb_cost_df_19_23[cols]
comb_cost_df_24    = comb_cost_df_24[cols]

In [14]:
comb_cost_df_19_23 = comb_cost_df_19_23.drop_duplicates(subset = ['bid_id','Year'])
comb_cost_df_24    = comb_cost_df_24.drop_duplicates(subset = ['bid_id','Year'])

In [17]:
comb_cost_final_input = pd.concat([comb_cost_df_24,comb_cost_df_19_23],ignore_index=True)

In [18]:
comb_cost_final_input

,bid_id,Year,Monthly_Consolidated_Premium_C_D,PartC_Premium,PartD_Premium,$0_Premium,Grand_Total_OOPC,In_network_MOOP_Amount,Type of Medicare Health Plan_HMO,Type of Medicare Health Plan_HMO_POS,...,b7k_Opioid_OON,b8a_Outpatient_Procedures_Tests_Lab_OON,b8b_Outpatient_X_Rays_OON,b9a_Outpatient_Hospital_OON,b9b_Ambulatory_Surgical_Center_OON,b9c_Substance_Abuse_OON,b9d_Outpatient_Blood_Services_OON,b1a_inpat_acute_OON,b1b_inpat_mental_OON,b2_snf_OON
0,H1848_001_0,2024,0.0,0.0,0.0,1,279.602329,7350.0,0,0,...,0.00028,20.870922,20.829694,20.870922,13.241101,0.672368,36.307623,188.651673,0.001721,1.240000e-07
1,H3080_001_0,2024,0.0,0.0,0.0,1,175.086735,4500.0,1,0,...,0.18430,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.847860,0.402541,1.244729e-02
2,H1848_002_0,2024,0.0,0.0,0.0,1,223.395424,5500.0,0,0,...,0.00021,15.653192,15.622270,15.653192,9.930826,0.504276,27.230717,188.651673,0.001721,1.240000e-07
3,H3080_002_0,2024,0.0,0.0,0.0,1,247.170625,6000.0,1,0,...,0.18430,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.847860,0.402541,1.244729e-02
4,H0432_003_0,2024,0.0,0.0,0.0,1,176.054522,4900.0,0,1,...,0.18430,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.847860,0.402541,1.244729e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28140,H5496_011_0,2023,0.0,0.0,0.0,1,240.120771,2999.0,1,0,...,0.18430,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.800000,0.402500,1.244700e-02
28141,H1099_001_0,2023,95.0,26.0,69.0,0,230.212038,3450.0,0,1,...,0.00014,10.435461,10.414847,10.435461,6.620551,0.336184,494.952454,8.497823,0.000077,5.590000e-09
28142,H1099_006_0,2023,31.0,0.0,31.0,0,245.147076,4200.0,1,0,...,0.18430,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.800000,0.402500,1.244700e-02
28143,H1099_014_0,2023,0.0,0.0,0.0,1,256.337808,4900.0,1,0,...,0.18430,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.800000,0.402500,1.244700e-02


In [19]:
df_new = comb_cost_final_input.drop(columns = ['bid_id','Year'])

In [20]:
scaler = StandardScaler()
df3 = scaler.fit_transform(df_new)
df_scaled = pd.DataFrame(df3,columns = df_new.columns)

In [21]:
weighted_data = df_new

In [22]:
weighted_data = df_scaled.copy(deep=True)

In [23]:
bene_weights

,Benefit_Names,Benefit_Weights,Benefit_Weights_New,Benefit_Weights_New_1,Benefit_Weights_New_2,Benefit_Weights_New_3,Benefit_Weights_New_4,Benefit_Weights_New_5
0,Monthly_Consolidated_Premium_C_D,10.00,5,5.0,2.50,2.0,5.0,5.0
1,PartC_Premium,10.00,5,5.0,2.50,2.0,5.0,5.0
2,PartD_Premium,10.00,5,5.0,2.50,2.0,2.0,2.0
3,$0_Premium,10.00,5,5.0,2.50,2.0,5.0,5.0
4,Grand_Total_OOPC,5.00,3,4.0,2.25,1.8,2.0,2.0
...,...,...,...,...,...,...,...,...
190,b9c_Substance_Abuse_OON,0.45,1,0.8,0.80,1.0,1.0,1.0
191,b9d_Outpatient_Blood_Services_OON,0.45,1,0.8,0.80,1.0,1.0,1.0
192,b1a_inpat_acute_OON,0.45,1,0.8,0.80,1.0,1.0,1.0
193,b1b_inpat_mental_OON,0.45,1,0.8,0.80,1.0,1.0,1.0


In [24]:
weighted_data

,Monthly_Consolidated_Premium_C_D,PartC_Premium,PartD_Premium,$0_Premium,Grand_Total_OOPC,In_network_MOOP_Amount,Type of Medicare Health Plan_HMO,Type of Medicare Health Plan_HMO_POS,Type of Medicare Health Plan_PPO,Annual_Drug_Deductible,...,b7k_Opioid_OON,b8a_Outpatient_Procedures_Tests_Lab_OON,b8b_Outpatient_X_Rays_OON,b9a_Outpatient_Hospital_OON,b9b_Ambulatory_Surgical_Center_OON,b9c_Substance_Abuse_OON,b9d_Outpatient_Blood_Services_OON,b1a_inpat_acute_OON,b1b_inpat_mental_OON,b2_snf_OON
0,-0.555701,-0.389531,-0.386699,0.825471,1.544739,1.106374,-1.055584,-0.363385,1.442102,0.629163,...,-1.401677,-1.226175,-1.249947,-1.248243,-1.225779,-1.256876,-1.306004,-1.258097,-1.264037,-1.335388
1,-0.555701,-0.389531,-0.386699,0.825471,-0.184679,-0.416660,0.947343,-0.363385,-0.693432,-0.683567,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790908,0.788004,0.779158
2,-0.555701,-0.389531,-0.386699,0.825471,0.614684,0.117738,-1.055584,-0.363385,1.442102,-0.683567,...,-1.402483,-1.245681,-1.266869,-1.267908,-1.272195,-1.263257,-1.318341,-1.258097,-1.264037,-1.335388
3,-0.555701,-0.389531,-0.386699,0.825471,1.008092,0.384937,0.947343,-0.363385,-0.693432,0.629163,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790908,0.788004,0.779158
4,-0.555701,-0.389531,-0.386699,0.825471,-0.168665,-0.202901,-1.055584,2.751900,-0.693432,-0.683567,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790908,0.788004,0.779158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28140,-0.555701,-0.389531,-0.386699,0.825471,0.891438,-1.218791,0.947343,-0.363385,-0.693432,0.532816,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790905,0.787795,0.779110
28141,1.510770,0.392216,0.003474,-1.211429,0.727478,-0.977778,-1.055584,2.751900,-0.693432,-0.683567,...,-1.403288,-1.265186,-1.283791,-1.287574,-1.318610,-1.269638,-0.682647,-1.266831,-1.272453,-1.335408
28142,0.118621,-0.389531,-0.211404,-1.211429,0.974608,-0.576979,0.947343,-0.363385,-0.693432,-0.683567,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790905,0.787795,0.779110
28143,-0.555701,-0.389531,-0.386699,0.825471,1.159781,-0.202901,0.947343,-0.363385,-0.693432,-0.683567,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790905,0.787795,0.779110


In [25]:
for index, row in bene_weights.iterrows():
    col_name  = bene_weights.at[index,'Benefit_Names']
    weight    = bene_weights.at[index,'Benefit_Weights_New_4']
    weighted_data[col_name] = weighted_data[col_name] * weight

In [27]:
df_scaled

,Monthly_Consolidated_Premium_C_D,PartC_Premium,PartD_Premium,$0_Premium,Grand_Total_OOPC,In_network_MOOP_Amount,Type of Medicare Health Plan_HMO,Type of Medicare Health Plan_HMO_POS,Type of Medicare Health Plan_PPO,Annual_Drug_Deductible,...,b7k_Opioid_OON,b8a_Outpatient_Procedures_Tests_Lab_OON,b8b_Outpatient_X_Rays_OON,b9a_Outpatient_Hospital_OON,b9b_Ambulatory_Surgical_Center_OON,b9c_Substance_Abuse_OON,b9d_Outpatient_Blood_Services_OON,b1a_inpat_acute_OON,b1b_inpat_mental_OON,b2_snf_OON
0,-0.555701,-0.389531,-0.386699,0.825471,1.544739,1.106374,-1.055584,-0.363385,1.442102,0.629163,...,-1.401677,-1.226175,-1.249947,-1.248243,-1.225779,-1.256876,-1.306004,-1.258097,-1.264037,-1.335388
1,-0.555701,-0.389531,-0.386699,0.825471,-0.184679,-0.416660,0.947343,-0.363385,-0.693432,-0.683567,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790908,0.788004,0.779158
2,-0.555701,-0.389531,-0.386699,0.825471,0.614684,0.117738,-1.055584,-0.363385,1.442102,-0.683567,...,-1.402483,-1.245681,-1.266869,-1.267908,-1.272195,-1.263257,-1.318341,-1.258097,-1.264037,-1.335388
3,-0.555701,-0.389531,-0.386699,0.825471,1.008092,0.384937,0.947343,-0.363385,-0.693432,0.629163,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790908,0.788004,0.779158
4,-0.555701,-0.389531,-0.386699,0.825471,-0.168665,-0.202901,-1.055584,2.751900,-0.693432,-0.683567,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790908,0.788004,0.779158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28140,-0.555701,-0.389531,-0.386699,0.825471,0.891438,-1.218791,0.947343,-0.363385,-0.693432,0.532816,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790905,0.787795,0.779110
28141,1.510770,0.392216,0.003474,-1.211429,0.727478,-0.977778,-1.055584,2.751900,-0.693432,-0.683567,...,-1.403288,-1.265186,-1.283791,-1.287574,-1.318610,-1.269638,-0.682647,-1.266831,-1.272453,-1.335408
28142,0.118621,-0.389531,-0.211404,-1.211429,0.974608,-0.576979,0.947343,-0.363385,-0.693432,-0.683567,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790905,0.787795,0.779110
28143,-0.555701,-0.389531,-0.386699,0.825471,1.159781,-0.202901,0.947343,-0.363385,-0.693432,-0.683567,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790905,0.787795,0.779110


In [28]:
weighted_data

,Monthly_Consolidated_Premium_C_D,PartC_Premium,PartD_Premium,$0_Premium,Grand_Total_OOPC,In_network_MOOP_Amount,Type of Medicare Health Plan_HMO,Type of Medicare Health Plan_HMO_POS,Type of Medicare Health Plan_PPO,Annual_Drug_Deductible,...,b7k_Opioid_OON,b8a_Outpatient_Procedures_Tests_Lab_OON,b8b_Outpatient_X_Rays_OON,b9a_Outpatient_Hospital_OON,b9b_Ambulatory_Surgical_Center_OON,b9c_Substance_Abuse_OON,b9d_Outpatient_Blood_Services_OON,b1a_inpat_acute_OON,b1b_inpat_mental_OON,b2_snf_OON
0,-2.778507,-1.947656,-0.773398,4.127356,3.089477,5.531868,-5.277921,-1.816927,7.210511,1.258327,...,-1.401677,-1.226175,-1.249947,-1.248243,-1.225779,-1.256876,-1.306004,-1.258097,-1.264037,-1.335388
1,-2.778507,-1.947656,-0.773398,4.127356,-0.369359,-2.083301,4.736714,-1.816927,-3.467161,-1.367133,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790908,0.788004,0.779158
2,-2.778507,-1.947656,-0.773398,4.127356,1.229368,0.588688,-5.277921,-1.816927,7.210511,-1.367133,...,-1.402483,-1.245681,-1.266869,-1.267908,-1.272195,-1.263257,-1.318341,-1.258097,-1.264037,-1.335388
3,-2.778507,-1.947656,-0.773398,4.127356,2.016183,1.924683,4.736714,-1.816927,-3.467161,1.258327,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790908,0.788004,0.779158
4,-2.778507,-1.947656,-0.773398,4.127356,-0.337331,-1.014505,-5.277921,13.759498,-3.467161,-1.367133,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790908,0.788004,0.779158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28140,-2.778507,-1.947656,-0.773398,4.127356,1.782876,-6.093956,4.736714,-1.816927,-3.467161,1.065633,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790905,0.787795,0.779110
28141,7.553852,1.961082,0.006948,-6.057146,1.454956,-4.888889,-5.277921,13.759498,-3.467161,-1.367133,...,-1.403288,-1.265186,-1.283791,-1.287574,-1.318610,-1.269638,-0.682647,-1.266831,-1.272453,-1.335408
28142,0.593105,-1.947656,-0.422808,-6.057146,1.949216,-2.884897,4.736714,-1.816927,-3.467161,-1.367133,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790905,0.787795,0.779110
28143,-2.778507,-1.947656,-0.773398,4.127356,2.319562,-1.014505,4.736714,-1.816927,-3.467161,-1.367133,...,0.715887,0.798435,0.795392,0.792980,0.792921,0.793402,0.787307,0.790905,0.787795,0.779110


# Results from the past iterations

From our past analysis we have got the number of clusters as 11. So we can directly use that result.

In [30]:
X = weighted_data.to_numpy()
kmeans = KMeans(n_clusters=11, random_state=42) # taking clusters as 6, which we obtained from clusetring analysis
kmeans.fit(X)

# Get the cluster assignments for each data point
labels = kmeans.labels_

# add cluster assignments back to data
comb_cost_final_input['clusters']=labels

In [31]:
comb_cost_final_input

,bid_id,Year,Monthly_Consolidated_Premium_C_D,PartC_Premium,PartD_Premium,$0_Premium,Grand_Total_OOPC,In_network_MOOP_Amount,Type of Medicare Health Plan_HMO,Type of Medicare Health Plan_HMO_POS,...,b8a_Outpatient_Procedures_Tests_Lab_OON,b8b_Outpatient_X_Rays_OON,b9a_Outpatient_Hospital_OON,b9b_Ambulatory_Surgical_Center_OON,b9c_Substance_Abuse_OON,b9d_Outpatient_Blood_Services_OON,b1a_inpat_acute_OON,b1b_inpat_mental_OON,b2_snf_OON,clusters
0,H1848_001_0,2024,0.0,0.0,0.0,1,279.602329,7350.0,0,0,...,20.870922,20.829694,20.870922,13.241101,0.672368,36.307623,188.651673,0.001721,1.240000e-07,2
1,H3080_001_0,2024,0.0,0.0,0.0,1,175.086735,4500.0,1,0,...,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.847860,0.402541,1.244729e-02,1
2,H1848_002_0,2024,0.0,0.0,0.0,1,223.395424,5500.0,0,0,...,15.653192,15.622270,15.653192,9.930826,0.504276,27.230717,188.651673,0.001721,1.240000e-07,6
3,H3080_002_0,2024,0.0,0.0,0.0,1,247.170625,6000.0,1,0,...,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.847860,0.402541,1.244729e-02,9
4,H0432_003_0,2024,0.0,0.0,0.0,1,176.054522,4900.0,0,1,...,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.847860,0.402541,1.244729e-02,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28140,H5496_011_0,2023,0.0,0.0,0.0,1,240.120771,2999.0,1,0,...,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.800000,0.402500,1.244700e-02,4
28141,H1099_001_0,2023,95.0,26.0,69.0,0,230.212038,3450.0,0,1,...,10.435461,10.414847,10.435461,6.620551,0.336184,494.952454,8.497823,0.000077,5.590000e-09,0
28142,H1099_006_0,2023,31.0,0.0,31.0,0,245.147076,4200.0,1,0,...,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.800000,0.402500,1.244700e-02,5
28143,H1099_014_0,2023,0.0,0.0,0.0,1,256.337808,4900.0,1,0,...,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.800000,0.402500,1.244700e-02,7


In [32]:
all_benefits = list(bene_weights['Benefit_Names']) 

In [34]:
clust_res = comb_cost_final_input.groupby(['clusters'], as_index=False, sort=True)[all_benefits].mean()
clust_res

,clusters,Monthly_Consolidated_Premium_C_D,PartC_Premium,PartD_Premium,$0_Premium,Grand_Total_OOPC,In_network_MOOP_Amount,Type of Medicare Health Plan_HMO,Type of Medicare Health Plan_HMO_POS,Type of Medicare Health Plan_PPO,...,b7k_Opioid_OON,b8a_Outpatient_Procedures_Tests_Lab_OON,b8b_Outpatient_X_Rays_OON,b9a_Outpatient_Hospital_OON,b9b_Ambulatory_Surgical_Center_OON,b9c_Substance_Abuse_OON,b9d_Outpatient_Blood_Services_OON,b1a_inpat_acute_OON,b1b_inpat_mental_OON,b2_snf_OON
0,0,67.346413,32.669395,83.455830,0.000000,202.510424,5068.553812,0.000000,1.000000,0.000000,...,0.064027,69.657726,96.154941,96.540421,30.495042,9.125382,457.024291,7156.884547,0.085959,0.003635
1,1,3.569561,0.297843,20.807164,0.864181,145.037730,2840.640011,0.988282,0.000000,0.000000,...,0.184300,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.809228,0.402508,0.012447
2,2,0.077233,0.019627,550.382434,0.996075,132.538598,5790.157026,0.000000,0.006869,0.971541,...,0.012474,14.784147,19.129053,17.172924,11.541431,0.621831,36.026160,23.965165,0.000363,0.000325
3,3,178.680093,136.568663,205.869596,0.000000,151.586447,3543.689736,0.323484,0.123639,0.313375,...,0.118265,317.385644,368.197710,319.840502,91.149006,30.902715,927.934090,24323.688217,0.232488,0.007549
4,4,0.148935,0.062698,0.086237,0.993446,187.915934,7518.824036,0.890224,0.000000,0.000000,...,0.184300,562.460254,650.239650,562.460254,157.210977,54.684254,1576.493004,42455.810743,0.402509,0.012447
5,5,44.086952,16.443143,65.153690,0.000000,200.456629,5606.084286,0.970952,0.000000,0.000000,...,0.184300,560.861502,650.239650,562.460254,157.210977,54.684254,1570.111961,42455.806416,0.402505,0.012447
6,6,0.000000,0.000000,0.000000,1.000000,202.130074,5991.308499,0.000000,0.000000,0.998961,...,0.012879,13.976832,19.345414,19.830469,11.640277,0.599302,39.282842,91.769877,0.001057,0.000381
7,7,0.000000,0.000000,57.591186,1.000000,201.042784,5804.831020,0.992208,0.000000,0.000000,...,0.184300,562.364080,650.239650,562.460254,157.210977,54.684254,1576.109149,42455.806654,0.402506,0.012447
8,8,53.450186,21.952809,48.943875,0.000000,206.379990,5841.827273,0.000000,0.000000,0.961148,...,0.029456,13.409716,17.920874,16.852779,10.858390,0.607028,48.074264,123.914760,0.001257,0.000329
9,9,0.000000,0.000000,103.021042,1.000000,168.419602,4345.618999,0.942799,0.041879,0.015322,...,0.178319,544.363334,629.458833,544.668699,152.241429,52.911645,1530.194955,40981.466567,0.389363,0.012052


In [36]:
clust_res_year = comb_cost_final_input.groupby(['clusters','Year'], as_index=False, sort=True)[all_benefits].mean()
clust_res_year

,clusters,Year,Monthly_Consolidated_Premium_C_D,PartC_Premium,PartD_Premium,$0_Premium,Grand_Total_OOPC,In_network_MOOP_Amount,Type of Medicare Health Plan_HMO,Type of Medicare Health Plan_HMO_POS,...,b7k_Opioid_OON,b8a_Outpatient_Procedures_Tests_Lab_OON,b8b_Outpatient_X_Rays_OON,b9a_Outpatient_Hospital_OON,b9b_Ambulatory_Surgical_Center_OON,b9c_Substance_Abuse_OON,b9d_Outpatient_Blood_Services_OON,b1a_inpat_acute_OON,b1b_inpat_mental_OON,b2_snf_OON
0,0,2019,71.539850,39.280451,98.133083,0.000000,181.097948,4929.699248,0.0,1.0,...,0.184300,32.721737,54.144032,57.320928,19.029722,7.834149,423.403492,4168.322753,0.057751,0.002951
1,0,2020,69.628986,34.675362,84.613043,0.000000,212.659275,5018.115942,0.0,1.0,...,0.052571,88.639556,110.925950,99.252032,30.042867,10.351190,498.548351,8317.726555,0.093506,0.003616
2,0,2021,68.115789,36.186842,105.468421,0.000000,175.111645,5070.394737,0.0,1.0,...,0.027189,38.889586,52.514207,47.844332,17.093401,6.583272,384.451733,3642.250807,0.055793,0.002615
3,0,2022,64.490278,31.039583,95.867361,0.000000,178.399931,5044.791667,0.0,1.0,...,0.026100,40.499643,54.432864,49.344812,18.299462,5.773084,394.697598,3847.344968,0.050492,0.002596
4,0,2023,67.673810,28.718452,66.122024,0.000000,232.828294,5069.761905,0.0,1.0,...,0.056415,117.651022,161.286890,168.441862,50.701804,12.748176,595.239148,12145.825760,0.131919,0.004837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,10,2020,5.094152,1.075439,84.170760,0.818713,204.132749,4652.046784,0.0,1.0,...,0.112232,317.875016,376.984570,328.006162,94.545673,31.891887,1066.443024,24588.693854,0.230781,0.007336
62,10,2021,5.545872,0.374771,82.083945,0.811927,157.703945,4779.587156,0.0,1.0,...,0.105016,300.538654,353.364634,303.771702,86.839231,28.851054,1024.323493,23379.063807,0.212478,0.006995
63,10,2022,3.639414,0.275570,72.373616,0.876221,163.563355,4858.785016,0.0,1.0,...,0.122004,355.910324,414.564442,355.602072,101.866307,34.405945,1141.854571,26985.804456,0.254560,0.008140
64,10,2023,7.047627,0.291930,62.464557,0.773734,202.605619,5065.424051,0.0,1.0,...,0.149693,446.247336,520.679272,453.469099,127.793911,43.030145,1322.780692,33663.756716,0.321707,0.010194


In [37]:
comb_cost_final_input.to_csv(r'C:\Users\Vivek.Sujan\Desktop\DS Assignment\Plan Segmentation 2024 Plans\Results\Final_Allyears_Combined\Plan_Segment_Cluster_AllYears.csv',index=False)

In [38]:
clust_res.to_csv(r'C:\Users\Vivek.Sujan\Desktop\DS Assignment\Plan Segmentation 2024 Plans\Results\Final_Allyears_Combined\Plan_Segment_Cluster_Avg_AllYears_Combined.csv',index=False)

In [40]:
clust_res_year.to_csv(r'C:\Users\Vivek.Sujan\Desktop\DS Assignment\Plan Segmentation 2024 Plans\Results\Final_Allyears_Combined\Plan_Segment_Cluster_Avg_AllYears_Separate.csv',index=False)